In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing


In [2]:
botnet_df = pd.read_csv("botnet_sample.csv")

In [3]:
#df = botnet_df.sample(n = 100000)

In [4]:
#df.to_csv(r"./botnet_sample.csv", index = False)

In [5]:
botnet_df.head()


,MI_dir_L0.1_weight,MI_dir_L0.1_mean,MI_dir_L0.1_variance,H_L0.1_weight,H_L0.1_mean,H_L0.1_variance,HH_L0.1_weight,HH_L0.1_mean,HH_L0.1_std,HH_L0.1_magnitude,...,HpHp_L0.1_mean,HpHp_L0.1_std,HpHp_L0.1_magnitude,HpHp_L0.1_radius,HpHp_L0.1_covariance,HpHp_L0.1_pcc,Device_Name,Attack,Attack_subType,label
0,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,1.000000,60.000000,0.000000,60.000000,...,60.0,0.0,60.00000,0.0,0.000000e+00,0.0,Philips_B120N10_Baby_Monitor,gafgyt,tcp,0
1,6359.849398,433.330869,49454.689613,6359.849398,433.330869,49454.689613,1.000000,60.000000,0.000000,60.000000,...,60.0,0.0,60.00000,0.0,0.000000e+00,0.0,Ecobee_Thermostat,mirai,ack,0
2,5777.340832,428.923288,50516.327209,5777.340832,428.923288,50516.327209,4211.512980,565.934568,5.753625,565.934568,...,566.0,0.0,566.00000,0.0,0.000000e+00,0.0,SimpleHome_XCS7_1002_WHT_Security_Camera,mirai,ack,0
3,1.733683,282.162556,10627.688840,1.733683,282.162556,10627.688840,1.213257,330.000000,0.000000,431.490440,...,330.0,0.0,431.49044,0.0,-4.260000e-122,0.0,SimpleHome_XCS7_1003_WHT_Security_Camera,Normal,Normal,1
4,5985.352612,427.188247,46548.123891,5985.352612,427.188247,46548.123891,4449.260817,553.914475,6.499374,553.914475,...,554.0,0.0,554.00000,0.0,0.000000e+00,0.0,SimpleHome_XCS7_1002_WHT_Security_Camera,mirai,udp,0


In [6]:
print(botnet_df.shape)

(100000, 27)


In [7]:
del botnet_df["Device_Name"]
del botnet_df["Attack_subType"]
del botnet_df["label"]

In [8]:
# We deleted other botnets beacuse they were too infrequent compared to two major ones.
botnet_df = botnet_df.loc[botnet_df['Attack'].isin(["gafgyt", "mirai"])]

In [9]:
# Changed categorical data to numeric type so we can calculate correlations.
botnet_df['Attack'].replace(['gafgyt', 'mirai'],
                        [0, 1], inplace=True)

In [10]:
scaler = preprocessing.MinMaxScaler()
d = scaler.fit_transform(botnet_df)
botnet_df = pd.DataFrame(d, columns = botnet_df.columns)

In [11]:
array = botnet_df.values
y = array[:,23]
X = botnet_df
del X["Attack"]
num_feats = 10

In [12]:
feature_name = list(X.columns)

In [13]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [14]:
def cor_selector(X, y,num_feats):
    cor_list = []
    feature_name = X.columns.tolist()
    # calculate the correlation with y for each feature
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i], y)[0, 1]
        cor_list.append(cor)
    # replace NaN with 0
    cor_list = [0 if np.isnan(i) else i for i in cor_list]
    # feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(cor_list))[-num_feats:]].columns.tolist()
    # feature selection? 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_name]
    return cor_support, cor_feature
cor_support, cor_feature = cor_selector(X, y,num_feats)
print(str(len(cor_feature)), 'selected features')

10 selected features


In [15]:
cor_feature

['HH_L0.1_magnitude',
 'HpHp_L0.1_magnitude',
 'HpHp_L0.1_mean',
 'HH_L0.1_mean',
 'H_L0.1_weight',
 'MI_dir_L0.1_weight',
 'H_L0.1_variance',
 'MI_dir_L0.1_variance',
 'MI_dir_L0.1_mean',
 'H_L0.1_mean']

In [16]:
X_norm = preprocessing.MinMaxScaler().fit_transform(X)
chi_selector = SelectKBest(chi2, k=num_feats)
chi_selector.fit(X_norm, y)
chi_support = chi_selector.get_support()
chi_feature = X.loc[:,chi_support].columns.tolist()
print(str(len(chi_feature)), 'selected features')

10 selected features


In [17]:
chi_feature

['MI_dir_L0.1_weight',
 'MI_dir_L0.1_mean',
 'MI_dir_L0.1_variance',
 'H_L0.1_weight',
 'H_L0.1_mean',
 'H_L0.1_variance',
 'HH_L0.1_mean',
 'HH_L0.1_magnitude',
 'HpHp_L0.1_mean',
 'HpHp_L0.1_magnitude']

In [18]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=num_feats, step=10, verbose=5)
rfe_selector.fit(X_norm, y)

Fitting estimator with 23 features.
Fitting estimator with 13 features.


RFE(estimator=LogisticRegression(), n_features_to_select=10, step=10, verbose=5)

In [19]:
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')

10 selected features


In [20]:
rfe_feature

['MI_dir_L0.1_weight',
 'MI_dir_L0.1_mean',
 'H_L0.1_weight',
 'HH_L0.1_weight',
 'HH_L0.1_magnitude',
 'HH_L0.1_radius',
 'HH_L0.1_pcc',
 'HH_jit_L0.1_weight',
 'HH_jit_L0.1_variance',
 'HpHp_L0.1_std']

In [21]:
pd.set_option('display.max_rows', None)
# put all selection together
feature_selection_df = pd.DataFrame({'Feature':feature_name, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(23)

/Users/bakolas/opt/anaconda3/envs/geopandas/lib/python3.7/site-packages/numpy/core/fromnumeric.py:84: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return reduction(axis=axis, out=out, **passkwargs)


,Feature,Pearson,Chi-2,RFE,Total
1,MI_dir_L0.1_weight,True,True,True,3
2,MI_dir_L0.1_mean,True,True,True,3
3,H_L0.1_weight,True,True,True,3
4,HH_L0.1_magnitude,True,True,True,3
5,MI_dir_L0.1_variance,True,True,False,2
6,HpHp_L0.1_mean,True,True,False,2
7,HpHp_L0.1_magnitude,True,True,False,2
8,H_L0.1_variance,True,True,False,2
9,H_L0.1_mean,True,True,False,2
10,HH_L0.1_mean,True,True,False,2
